In [41]:
import torch

In [42]:
passwords = open('rockyou.txt', 'r',encoding='latin1').read().splitlines()

In [43]:
passwords[1000:1010] # some nice passwords lol

['trustno1',
 'sexylady',
 '14344',
 'autumn',
 'mendoza',
 'animals',
 'perfect',
 'mariel',
 'bullshit',
 'bitches']

In [44]:
len(passwords)

14346469

In [45]:
# some cleaning, filtering out unusual, rare and special characters, also capital letters for simplification
allowed = set("abcdefghijklmnopqrstuvwxyz0123456789")
filtered_passwords = []
for p in passwords:
    if all(char in allowed for char in p):
        filtered_passwords.append(p)

In [46]:
len(filtered_passwords)

12147978

In [47]:
chars = sorted(list(set(''.join(filtered_passwords))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['<s>'] = 0
stoi['</s>'] = len(stoi)
itos = {i: ch for ch, i in stoi.items()}
stoi

{'0': 1,
 '1': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10,
 'a': 11,
 'b': 12,
 'c': 13,
 'd': 14,
 'e': 15,
 'f': 16,
 'g': 17,
 'h': 18,
 'i': 19,
 'j': 20,
 'k': 21,
 'l': 22,
 'm': 23,
 'n': 24,
 'o': 25,
 'p': 26,
 'q': 27,
 'r': 28,
 's': 29,
 't': 30,
 'u': 31,
 'v': 32,
 'w': 33,
 'x': 34,
 'y': 35,
 'z': 36,
 '<s>': 0,
 '</s>': 37}

In [63]:
# creating training set
X_train = []
y_train = []

for p in filtered_passwords[:500000]: # training with part of dataset to avoid memory allocation issues 
  chs = ['<s>'] + list(p) + ['</s>'] # <s> and </s> are respectively special start/end indicators
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    X_train.append(ix1)
    y_train.append(ix2)
    
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)


In [64]:
X_train

tensor([ 0,  2,  3,  ..., 25, 24, 11])

In [65]:
y_train

tensor([ 2,  3,  4,  ..., 24, 11, 37])

In [66]:
# to train NN, lets try batch processing
# initialize NN weights randomly
W = torch.randn((38,38),requires_grad=True)

In [69]:
import torch.nn.functional as F
batch_size = 5000  
n_batches = len(X_train) // batch_size

losses = []
for k in range(100):  # training iterations
    total_loss = 0
    for i in range(n_batches):
        # get batch
        start_idx = i * batch_size
        end_idx = start_idx + batch_size
        X_batch = X_train[start_idx:end_idx]
        y_batch = y_train[start_idx:end_idx]
        
        # one hot encoding the input batch to the neural network
        X_enc = F.one_hot(X_batch, num_classes=38).float()
        
        # forward pass
        logits = X_enc @ W
        counts = logits.exp()
        probs = counts / counts.sum(1, keepdims=True)
        loss = -probs[torch.arange(batch_size), y_batch].log().mean() + 0.01*(W**2).mean()
        
        # backward pass
        W.grad = None
        loss.backward()
        
        # update
        W.data += -50 * W.grad
        total_loss += loss.item()
    
    if k % 5 == 0:
        print(f'Iteration {k}, Average Loss: {total_loss/n_batches}')
        

Iteration 0, Average Loss: 2.7854925304969433
Iteration 5, Average Loss: 2.786347840038766
Iteration 10, Average Loss: 2.786347845129554
Iteration 15, Average Loss: 2.7863478473921264
Iteration 20, Average Loss: 2.7863478465436615
Iteration 25, Average Loss: 2.7863478389074796
Iteration 30, Average Loss: 2.7863478479577695
Iteration 35, Average Loss: 2.7863478448467323
Iteration 40, Average Loss: 2.786347840038766
Iteration 45, Average Loss: 2.7863478473921264
Iteration 50, Average Loss: 2.7863478465436615
Iteration 55, Average Loss: 2.7863478389074796
Iteration 60, Average Loss: 2.7863478479577695
Iteration 65, Average Loss: 2.7863478448467323
Iteration 70, Average Loss: 2.786347840038766
Iteration 75, Average Loss: 2.7863478473921264
Iteration 80, Average Loss: 2.7863478465436615
Iteration 85, Average Loss: 2.7863478389074796
Iteration 90, Average Loss: 2.7863478479577695
Iteration 95, Average Loss: 2.7863478448467323


In [68]:
# fun part :D
for i in range(10): # range is number of passwords sampled from NN
  
  out = []
  ix = 0
  while True:
    xenc = F.one_hot(torch.tensor([ix]), num_classes=38).float()
    logits = xenc @ W
    counts = logits.exp() 
    p = counts / counts.sum(1, keepdims = True) # probabilities for next character
    # ----------
    
    ix = torch.multinomial(p, num_samples=1, replacement=True).item()
    out.append(itos[ix])
    if ix == len(itos)-1:
      break
  print(''.join(out[:len(out)-1]))

ble1
bly1
banannkembelares
be14jetilaniton
buvy
buvelaneri3
bonk
betodlbesakl
belyraincast
blmowiqugtcasheteng2
